> Ce qui suit est une adaptation de : 

https://towardsdatascience.com/implementing-a-fully-convolutional-network-fcn-in-tensorflow-2-3c46fb61de3b

Avec :

https://github.com/himanshurawlani/fully_convolutional_network

# Création d'un FCN

In [104]:
import tensorflow as tf

In [105]:
def FCN_model(len_classes=5, dropout_rate=0.2):
    
    input = tf.keras.layers.Input(shape=(None, None, 3)) # Les tailles None permettent de prendre n'importe quelle taille d'image en entrée

    x = tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1)(input)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # x = tf.keras.layers.MaxPooling2D()(x)

    x = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # # Fully connected layer 1
    x = tf.keras.layers.Conv2D(filters=64, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # # Fully connected layer 2
    x = tf.keras.layers.Conv2D(filters=len_classes, kernel_size=1, strides=1)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    predictions = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=input, outputs=predictions)
    
    print(model.summary())
    print(f'Total number of layers: {len(model.layers)}')

    return model


In [106]:
FCN_model(len_classes=10, dropout_rate=0.2) # 162 classes car 162 champions différents

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 conv2d_59 (Conv2D)          (None, None, None, 32)    896       
                                                                 
 dropout_58 (Dropout)        (None, None, None, 32)    0         
                                                                 
 batch_normalization_58 (Bat  (None, None, None, 32)   128       
 chNormalization)                                                
                                                                 
 activation_64 (Activation)  (None, None, None, 32)    0         
                                                                 
 conv2d_60 (Conv2D)          (None, None, None, 64)    18496     
                                                           

### Création des images train et val 

In [107]:
import os, cv2
from shutil import copy2
import numpy as np
import tensorflow as tf 


def split_dataset(PATH_DOSS = './images_draft', DATASET_PATH = './dataset', train_images = 750, val_images = 60):
    # Specify path to the folder with all the images
    nom_champions = os.listdir(PATH_DOSS)

    # on enlève la fin des noms des fichiers en "_0.jpg" pour avoir juste le nom du champion
    for i in range(len(nom_champions)):
        nom_champions[i] = nom_champions[i][:-6]

    # On multiplie par 5 les images pour les avoir en plusieurs exemplaires
    noms_champions_final = nom_champions.copy()
    for i in range(4):
        noms_champions_bis = [nom_champions[x] + f"{i}" for x in range(len(nom_champions))]
        noms_champions_final += noms_champions_bis

    # On mélange les images
    np.random.shuffle(noms_champions_final)

    # On en récupère "train_images" pour l'entrainement et "val_images" pour la validation que l'on met dans le dossier dataset au bon endroit
    train_champions = noms_champions_final[:train_images]
    val_champions = noms_champions_final[train_images:train_images+val_images]

    # Specify path for copying the dataset into train and val sets
    os.makedirs(DATASET_PATH, exist_ok=True)

    # Creating train directory
    train_dir = os.path.join(DATASET_PATH, 'train')
    os.makedirs(train_dir, exist_ok=True)

    # Creating val directory
    val_dir = os.path.join(DATASET_PATH, 'val')
    os.makedirs(val_dir, exist_ok=True) 

    # On copie les images dans le dossier dataset
    for i in range(len(train_champions)):
        if train_champions[i] not in nom_champions:
            nom_image = train_champions[i][:-1] + '_0.jpg'
            copy2(PATH_DOSS + '/' + nom_image, DATASET_PATH + '/train/' + train_champions[i] + '.jpg')
        else :
            nom_image = train_champions[i] + '_0.jpg'
            copy2(PATH_DOSS + '/' + nom_image, DATASET_PATH + '/train/' + train_champions[i] + '4.jpg')

    for i in range(len(val_champions)):
        if val_champions[i] not in nom_champions:
            nom_image = val_champions[i][:-1] + '_0.jpg'
            copy2(PATH_DOSS + '/' + nom_image, DATASET_PATH + '/val/' + val_champions[i] + '.jpg')
        else :
            nom_image = val_champions[i] + '_0.jpg'
            copy2(PATH_DOSS + '/' + nom_image, DATASET_PATH + '/val/' + val_champions[i] + '4.jpg') 

In [108]:
split_dataset()

### Création de ce qui permet de faire le dataset avec les bons labels et les bonnes tailles

In [109]:
import os
import numpy as np
import cv2
from sklearn import preprocessing
import tensorflow as tf


class Generator(tf.keras.utils.Sequence):

    def __init__(self, DATASET_PATH, BATCH_SIZE=32, shuffle_images=True, image_min_side=24):
        """ Initialize Generator object.
        Args
            DATASET_PATH           : Path to folder containing individual folders named by their class names
            BATCH_SIZE             : The size of the batches to generate.
            shuffle_images         : If True, shuffles the images read from the DATASET_PATH
            image_min_side         : After resizing the minimum side of an image is equal to image_min_side.
        """

        self.batch_size = BATCH_SIZE
        self.shuffle_images = shuffle_images
        self.image_min_side = image_min_side
        self.load_image_paths_labels(DATASET_PATH)
        self.create_image_groups()
    
    def load_image_paths_labels(self, DATASET_PATH):
        

        nom_champions_gen = os.listdir('./images_draft')
        lb = preprocessing.LabelBinarizer()
        lb.fit(nom_champions_gen)
# On lit chaque image et on la met dans une liste, on rajoute avec son label en prenant garde de supprimer le numéro de l'image
        self.image_paths = []
        self.image_labels = []
        for champ in os.listdir(DATASET_PATH):
            champ_nom_temp = os.path.join(DATASET_PATH, champ)
            self.image_paths.append(os.path.join(DATASET_PATH, champ))
            self.image_labels.append(champ[:-5])
    
        self.image_labels = np.array(lb.transform(self.image_labels), dtype='float32')
        
        assert len(self.image_paths) == len(self.image_labels)

    def create_image_groups(self):
        if self.shuffle_images:
            # Randomly shuffle dataset
            seed = 4321
            np.random.seed(seed)
            np.random.shuffle(self.image_paths)
            np.random.seed(seed)
            np.random.shuffle(self.image_labels)

        # Divide image_paths and image_labels into groups of BATCH_SIZE
        self.image_groups = [[self.image_paths[x % len(self.image_paths)] for x in range(i, i + self.batch_size)]
                              for i in range(0, len(self.image_paths), self.batch_size)]
        self.label_groups = [[self.image_labels[x % len(self.image_labels)] for x in range(i, i + self.batch_size)]
                              for i in range(0, len(self.image_labels), self.batch_size)]

    def resize_image(self, img, min_side_len):

        h, w, c = img.shape

        # limit the min side maintaining the aspect ratio
        if min(h, w) < min_side_len:
            im_scale = float(min_side_len) / h if h < w else float(min_side_len) / w
        else:
            im_scale = 1.

        new_h = int(h * im_scale)
        new_w = int(w * im_scale)

        re_im = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_AREA)
        return re_im, new_h / h, new_w / w

    def load_images(self, image_group):

        images = []
        for image_path in image_group:
            img = cv2.imread(image_path)
            img_shape = len(img.shape)
            if img_shape == 2:
                img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
            elif img_shape == 4:
                img = cv2.cvtColor(img,cv2.COLOR_BGRA2RGB)
            elif img_shape == 3:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img, rh, rw = self.resize_image(img, self.image_min_side)
            images.append(img)

        return images

    def construct_image_batch(self, image_group):
        # get the max image shape
        max_shape = tuple(max(image.shape[x] for image in image_group) for x in range(3))

        # construct an image batch object
        image_batch = np.zeros((self.batch_size,) + max_shape, dtype='float32')

        # copy all images to the upper left part of the image batch object
        for image_index, image in enumerate(image_group):
            image_batch[image_index, :image.shape[0], :image.shape[1], :image.shape[2]] = image

        return image_batch
    
    def __len__(self):
        """
        Number of batches for generator.
        """

        return len(self.image_groups)

    def __getitem__(self, index):
        """
        Keras sequence method for generating batches.
        """
        image_group = self.image_groups[index]
        label_group = self.label_groups[index]
        images = self.load_images(image_group)
        image_batch = self.construct_image_batch(images)

        return np.array(image_batch), np.array(label_group)



In [110]:
BASE_PATH = 'dataset'
train_generator = Generator('dataset/train')
val_generator = Generator('dataset/val')
print(len(train_generator))
print(len(val_generator))
image_batch, label_group = train_generator.__getitem__(0)
print(image_batch.shape)
print(label_group.shape)

24
2
(32, 380, 380, 3)
(32, 162)


### train du modèle

In [116]:
import tensorflow as tf
import os

def train(model, train_generator, val_generator, epochs = 50):
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

    checkpoint_path = './snapshots'
    os.makedirs(checkpoint_path, exist_ok=True)
    model_path = os.path.join(checkpoint_path, 'model_epoch_{epoch:02d}_loss_{loss:.2f}_accuracy_{accuracy:.2f}_val_loss_{val_loss:.2f}_val_accuracy_{val_accuracy:.2f}.h5')
    
    history = model.fit_generator(generator=train_generator,
                                    steps_per_epoch=len(train_generator),
                                    epochs=epochs,
                                    callbacks=[tf.keras.callbacks.ModelCheckpoint(model_path, monitor='val_loss', save_best_only=True, verbose=1)],
                                    validation_data=val_generator,
                                    validation_steps=len(val_generator))

    return history
    

In [117]:
# Create FCN model
model = FCN_model(len_classes=162, dropout_rate=0.2)

# The below folders are created using utils.py
train_dir = 'dataset/train'
val_dir = 'dataset/val'

# If you get out of memory error try reducing the batch size
BATCH_SIZE=8
train_generator = Generator(train_dir, BATCH_SIZE, shuffle_images=True, image_min_side=24)
val_generator = Generator(val_dir, BATCH_SIZE, shuffle_images=True, image_min_side=24)

EPOCHS=50
history = train(model, train_generator, val_generator, epochs=EPOCHS)

Model: "model_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, None, None, 3)]   0         
                                                                 
 conv2d_87 (Conv2D)          (None, None, None, 32)    896       
                                                                 
 dropout_86 (Dropout)        (None, None, None, 32)    0         
                                                                 
 batch_normalization_86 (Bat  (None, None, None, 32)   128       
 chNormalization)                                                
                                                                 
 activation_92 (Activation)  (None, None, None, 32)    0         
                                                                 
 conv2d_88 (Conv2D)          (None, None, None, 64)    18496     
                                                          

None
Total number of layers: 30
Epoch 1/50


C:\Users\maila\AppData\Local\Temp\ipykernel_14796\1480894160.py:13: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


94/94 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 1: val_loss improved from inf to 0.00000, saving model to ./snapshots\model_epoch_01_loss_0.00_accuracy_0.00_val_loss_0.00_val_accuracy_0.00.h5
94/94 [==============================] - 380s 4s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/50
94/94 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0000e+00
Epoch 2: val_loss did not improve from 0.00000
94/94 [==============================] - 374s 4s/step - loss: 0.0000e+00 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/50
13/94 [===>..........................] - ETA: 5:13 - loss: 0.0000e+00 - accuracy: 0.0000e+00

KeyboardInterrupt: 